# Поиск ответа на вопрос в определенном тексте

## Подготовка среды

In [ ]:
# установка библиотек
! pip install -U transformers
! pip install -U accelerate

In [ ]:
# подключение к Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

## Подготовка модели

In [ ]:
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
device="cuda"
tokenizer = AutoTokenizer.from_pretrained("timpal0l/mdeberta-v3-base-squad2")

model = AutoModelForQuestionAnswering.from_pretrained("timpal0l/mdeberta-v3-base-squad2").to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/453 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/879 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

## Сохранение модели

In [ ]:
# сохранение в локальное хранилище
#model.save_pretrained("drive/MyDrive/Colab Notebooks/semantic_search_QA_telebot/qa_model")

Mounted at /content/drive


## Определение текста

In [ ]:
text = """

Гостиная Анны Павловны начала понемногу наполняться. Приехала высшая знать Петербурга, люди самые разнородные по возрастам и характерам, но одинаковые по обществу, в каком все жили; приехала дочь князя Василия, красавица Элен, заехавшая за отцом, чтобы с ним вместе ехать на праздник посланника. Она была в шифре и бальном платье. Приехала и известная, как la femme la plus sеduisante de Pеtersbourg[57 - самая обворожительная женщина в Петербурге.], молодая, маленькая княгиня Болконская, прошлую зиму вышедшая замуж и теперь не выезжавшая в большой свет по причине своей беременности, но ездившая еще на небольшие вечера. Приехал князь Ипполит, сын князя Василия, с Мортемаром, которого он представил; приехал и аббат Морио и многие другие. – Вы не видали еще, – или: – вы не знакомы с ma tante?[58 - моей тетушкой?] – говорила Анна Павловна приезжавшим гостям и весьма серьезно подводила их к маленькой старушке в высоких бантах, выплывшей из другой комнаты, как скоро стали приезжать гости, называла их по имени, медленно переводя глаза с гостя на ma tante, и потом отходила. Все гости совершали обряд приветствования никому не известной, никому не интересной и не нужной тетушке. Анна Павловна с грустным, торжественным участием следила за их приветствиями, молчаливо одобряя их. Ma tante каждому говорила в одних и тех же выражениях о его здоровье, о своем здоровье и о здоровье ее величества, которое нынче было, слава Богу, лучше. Все подходившие, из приличия не выказывая поспешности, с чувством облегчения исполненной тяжелой обязанности отходили от старушки, чтоб уж весь вечер ни разу не подойти к ней.
Молодая княгиня Болконская приехала с работой в шитом золотом бархатном мешке. Ее хорошенькая, с чуть черневшимися усиками верхняя губка была коротка по зубам, но тем милее она открывалась и тем еще милее вытягивалась иногда и опускалась на нижнюю. Как это бывает у вполне привлекательных женщин, недостаток ее – короткость губы и полуоткрытый рот – казались ее особенною, собственно ее красотой. Всем было весело смотреть на эту полную здоровья и живости хорошенькую будущую мать, так легко переносившую свое положение. Старикам и скучающим, мрачным молодым людям казалось, что они сами делаются похожи на нее, побыв и поговорив несколько времени с ней. Кто говорил с ней и видел при каждом слове ее светлую улыбочку и блестящие белые зубы, которые виднелись беспрестанно, тот думал, что он особенно нынче любезен. И это думал каждый. Маленькая княгиня, переваливаясь, маленькими быстрыми шажками обошла стол с рабочею сумочкой на руке и, весело оправляя платье, села на диван, около серебряного самовара, как будто все, что она ни делала, было partie de plaisir[59 - увеселение.] для нее и для всех ее окружавших.

– J’ai apportе mon ouvrage[60 - Я захватила работу.], – сказала она, развертывая свой ридикюль и обращаясь ко всем вместе.
– Смотрите, Annette, ne me jouez pas un mauvais tour, – обратилась она к хозяйке. – Vous m’avez еcrit, que c’еtait une toute petite soirеe; voyez comme je suis attifеe[61 - не сыграйте со мной злой шутки; вы мне писали, что у вас совсем маленький вечер. Видите, как я укутана.].

И она развела руками, чтобы показать свое, в кружевах, серенькое изящное платье, немного ниже грудей опоясанное широкою лентой.

– Soyez tranquille, Lise, vous serez toujours la plus jolie[62 - Будьте покойны, Лиза, вы все-таки будете лучше всех.], – отвечала Анна Павловна.
– Vous savez, mon mari m’abandonne, – продолжала она тем же тоном, обращаясь к генералу, – il va se faire tuer. Dites moi, pourquoi cette vilaine guerre[63 - Вы знаете, мой муж покидает меня. Идет на смерть. Скажите, зачем эта гадкая война.], – сказала она князю Василию и, не дожидаясь ответа, обратилась к дочери князя Василия, к красивой Элен.
– Quelle dеlicieuse personne, que cette petite princesse![64 - Что за милая особа, эта маленькая княгиня.] – сказал князь Василий тихо Анне Павловне.

Вскоре после маленькой княгини вошел массивный, толстый молодой человек с стриженою головой, в очках, светлых панталонах по тогдашней моде, с высоким жабо и в коричневом фраке. Этот толстый молодой человек был незаконный сын знаменитого екатерининского вельможи, графа Безухова, умиравшего теперь в Москве. Он нигде не служил еще, только что приехал из-за границы, где он воспитывался, и был первый раз в обществе. Анна Павловна приветствовала его поклоном, относящимся к людям самой низшей иерархии в ее салоне. Но, несмотря на это низшее по своему сорту приветствие, при виде вошедшего Пьера в лице Анны Павловны изобразилось беспокойство и страх, подобный тому, который выражается при виде чего-нибудь слишком огромного и несвойственного месту. Хотя действительно Пьер был несколько больше других мужчин в комнате, но этот страх мог относиться только к тому умному и вместе робкому, наблюдательному и естественному взгляду, отличавшему его от всех в этой гостиной.

– C'est bien aimable ? vous, monsieur Pierre, d'?tre venu voir une pauvre malade[65 - Очень мило с вашей стороны, мосье Пьер, что вы приехали навестить бедную больную.], – сказала ему Анна Павловна, испуганно переглядываясь с тетушкой, к которой она подводила его. Пьер пробурлил что-то непонятное и продолжал отыскивать что-то глазами. Он радостно, весело улыбнулся, кланяясь маленькой княгине, как близкой знакомой, и подошел к тетушке. Страх Анны Павловны был не напрасен, потому что Пьер, не дослушав речи тетушки о здоровье ее величества, отошел от нее, Анна Павловна испуганно остановила его словами:
– Вы не знаете аббата Морио? Он очень интересный человек… – сказала она.
– Да, я слышал про его план вечного мира, и это очень интересно, но едва ли возможно…
– Вы думаете?.. – сказала Анна Павловна, чтобы сказать что-нибудь и вновь обратиться к своим занятиям хозяйки дома, но Пьер сделал обратную неучтивость. Прежде он, не дослушав слов собеседницы, ушел; теперь он остановил своим разговором собеседницу, которой нужно было от него уйти. Он, нагнув голову и расставив большие ноги, стал доказывать Анне Павловне, почему он полагал, что план аббата был химера.
– Мы после поговорим, – сказала Анна Павловна улыбаясь.

И, отделавшись от молодого человека, не умеющего жить, она возвратилась к своим занятиям хозяйки дома и продолжала прислушиваться и приглядываться, готовая подать помощь на тот пункт, где ослабевал разговор. Как хозяин прядильной мастерской, посадив работников по местам, прохаживается по заведению, замечая неподвижность или непривычный, скрипящий, слишком громкий звук веретена, торопливо идет, сдерживает или пускает его в надлежащий ход, – так и Анна Павловна, прохаживаясь по своей гостиной, подходила к замолкнувшему или слишком много говорившему кружку и одним словом или перемещением опять заводила равномерную, приличную разговорную машину. Но среди этих забот все виден был в ней особенный страх за Пьера. Она заботливо поглядывала на него в то время, как он подошел послушать то, что говорилось около Мортемара, и отошел к другому кружку, где говорил аббат. Для Пьера, воспитанного за границей, этот вечер Анны Павловны был первый, который он видел в России. Он знал, что тут собрана вся интеллигенция Петербурга, и у него, как у ребенка в игрушечной лавке, разбегались глаза. Он все боялся пропустить умные разговоры, которые он может услыхать. Глядя на уверенные и изящные выражения лиц, собранных здесь, он все ждал чего-нибудь особенно умного. Наконец он подошел к Морио. Разговор показался ему интересен, и он остановился, ожидая случая высказать свои мысли, как это любят молодые люди.
"""

## Определение вопроса

In [ ]:
question = 'Кто приехал в гости к Анне Павловне?'
#question = 'К кому в гости приехала высшая знать Петербурга?'

## Обработка вопроса

In [ ]:
# Токенизация
tokenized = tokenizer.encode_plus(
   question, text,
   add_special_tokens=False
)

tokens = tokenizer.convert_ids_to_tokens(tokenized['input_ids'])

In [ ]:
# Общая длина каждого блока
max_chunk_length = 512
# Длина наложения
overlapped_length = 30

In [ ]:
# Длина вопроса в токенах
answer_tokens_length = tokenized.token_type_ids.count(0)
# Токены вопроса, закодированные числами
answer_input_ids = tokenized.input_ids[:answer_tokens_length]

In [ ]:
# Длина основного текста первого блока без наложения
first_context_chunk_length = max_chunk_length - answer_tokens_length
# Длина основного текста остальных блоков с наложением
context_chunk_length = max_chunk_length - answer_tokens_length - overlapped_length

In [ ]:
# Токены основного текста
context_input_ids = tokenized.input_ids[answer_tokens_length:]
# Основной текст первого блока
first = context_input_ids[:first_context_chunk_length]
# Основной текст остальных блоков
others = context_input_ids[first_context_chunk_length:]

# Если есть блоки кроме первого
# тогда обрабатываются все блоки
if len(others) > 0:
  # Кол-во нулевых токенов, для выравнивания последнего блока по длине
  padding_length = context_chunk_length - (len(others) % context_chunk_length)
  others += [0] * padding_length

  # Кол-во блоков и их длина без добавления наложения
  new_size = (
      len(others) // context_chunk_length,
      context_chunk_length
  )

  # Упаковка блоков
  new_context_input_ids = np.reshape(others, new_size)

  # Вычисление наложения
  overlappeds = new_context_input_ids[:, -overlapped_length:]
  # Добавление в наложения частей из первого блока
  overlappeds = np.insert(overlappeds, 0, first[-overlapped_length:], axis=0)
  # Удаление наложение из последнего блока, так как оно не нужно
  overlappeds = overlappeds[:-1]

  # Добавление наложения
  new_context_input_ids = np.c_[overlappeds, new_context_input_ids]
  # Добавление первого блока
  new_context_input_ids = np.insert(new_context_input_ids, 0, first, axis=0)

  # Добавление вопроса в каждый блок
  new_input_ids = np.c_[
    [answer_input_ids] * new_context_input_ids.shape[0],
    new_context_input_ids
  ]
# иначе обрабатывается только первый
else:
  # Кол-во нулевых токенов, для выравнивания блока по длине
  padding_length = first_context_chunk_length - (len(first) % first_context_chunk_length)
  # Добавление нулевых токенов
  new_input_ids = np.array(
    [answer_input_ids + first + [0] * padding_length]
  )

In [ ]:
# Кол-во блоков
count_chunks = new_input_ids.shape[0]

# Маска, разделяющая вопрос и текст
new_token_type_ids = [
  # вопрос блока
  [0] * answer_tokens_length
  # текст блока
  + [1] * (max_chunk_length - answer_tokens_length)
] * count_chunks

# Маска "внимания" модели на все токены, кроме нулевых в последнем блоке
new_attention_mask = (
  # во всех блоках, кроме последнего, "внимание" на все слова
  [[1] * max_chunk_length] * (count_chunks - 1)
  # в последнем блоке "внимание" только на ненулевые токены
  + [([1] * (max_chunk_length - padding_length)) + ([0] * padding_length)]
)

In [ ]:
# Токенизированный текст в виде блоков, упакованный в torch
new_tokenized = {
 'input_ids': torch.tensor(new_input_ids).to(device),
 'token_type_ids': torch.tensor(new_token_type_ids).to(device),
 'attention_mask': torch.tensor(new_attention_mask).to(device)
}
device = "cuda"
outputs = model(**new_tokenized)

In [ ]:
# Позиции в 2D списке токенов начала и конца наиболее вероятного ответа
# позиции одним числом
start_index = torch.argmax(outputs.start_logits)
end_index = torch.argmax(outputs.end_logits)

# Пересчёт позиций начала и конца ответа для 1D списка токенов
# = длина первого блока + (
#   позиция - длина первого блока
#   - длина ответов и отступов во всех блоках, кроме первого
# )
start_index = max_chunk_length + (
  start_index - max_chunk_length
  - (answer_tokens_length + overlapped_length)
  * (start_index // max_chunk_length)
)
end_index = max_chunk_length + (
  end_index - max_chunk_length
  - (answer_tokens_length + overlapped_length)
  * (end_index // max_chunk_length)
)

# Составление ответа
# если есть символ начала слова '▁', то он заменяется на пробел
answer = ''.join(
  [t.replace('▁', ' ') for t in tokens[start_index:end_index+1]]
)

In [ ]:
print('Вопрос:', question)
print('Ответ:', answer)

## Очистка кэша

In [ ]:
torch.cuda.empty_cache()